# Creador de un segmentador automático para nuestras células

In [5]:
import os
import skimage.io
import pandas as pd
import numpy as np
from skimage.filters import threshold_otsu, threshold_li
from skimage.morphology import erosion, dilation, disk
from skimage.measure import label, regionprops_table

# Creamos una función de segmentación
def hsc_segmentation(status, sample, reference, xpos, ypos, surrpix,l):
    # Carga el iterador
    xls = pd.ExcelFile("C:/Users/cdedi/Desktop/numpy_parser.xlsx")
    coord = xls.parse(index_col=None, na_values=['NA'])
    coord.columns = coord.columns.str.replace(' ', '')
    red_vector = [0] * 3
    segmented_object = [0] * 3
    for i in range(0,3,1):
        if (coord.at[j,"PositionZ"]+i) < 10:
            img_now = skimage.io.imread("C:/Users/cdedi/Desktop/Master images/" + status + "/" + sample + "/T00001C01Z00" + str(coord.at[j,"PositionZ"]+i) + ".tif")
        else:
            img_now = skimage.io.imread("C:/Users/cdedi/Desktop/Master images/" + status + "/" + sample + "/T00001C01Z0" + str(coord.at[j,"PositionZ"]+i) + ".tif")
        red_vector[i] = img_now[(ypos-surrpix):(ypos+surrpix),(xpos-surrpix):(xpos+surrpix)]
    # Unificamos las capas en un único array de Numpy para que sean un objeto 3D
    red_3d = np.array([red_vector[0],red_vector[1],red_vector[2]])
    # Generamos un elemento estructural 3D con scikit y numpy
    se_layer_major = disk(2)
    se_layer_minor = disk(1)
    filler = np.zeros((5,5))
    filler[1:4,1:4] = se_layer_minor
    SE_3D = np.array([filler,se_layer_major,filler])
    # Aplicamos operaciones morfológicas de dilatación y erosión
    dilated_mask = dilation(red_3d, selem=SE_3D)
    eroded_mask = erosion(dilated_mask, selem=SE_3D)
    # Habiendo tenido en cuenta la estructura 3D para las operaciones morfológicas, segmentamos cada plano de la estructura 3D por separado con Otsu
    for k in range(0,3,1):
        thresh = threshold_otsu(eroded_mask[k,:,:])
        segmented = eroded_mask[k,:,:] > thresh
        skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
        segmented_object[k] = segmented
    # Reconstruimos la máscara de segmentación 3D con numpy
    object_3d_segmented = np.array([segmented_object[0],segmented_object[1],segmented_object[2]])
    # Generamos unos labels de cada una de las partes de la máscara generada en el plano central para extraer su área
    segmented_labels = label(segmented_object[1])
    # Extraemos el área
    features = pd.DataFrame(regionprops_table(segmented_labels, properties=["area"]))
    area_matrix = features.loc[:,"area"]
    area_vector[l] = np.sum(area_matrix)
    # Guardamos todos los píxeles positivos para todos los planos como volumen
    volume_vector[l] = np.sum(object_3d_segmented)
    # Guardamos el Numpy
    array_vector[l] = object_3d_segmented.astype(int)
    # Trasladamos las características
    age_vector[l] = status
    sample_vector[l] = sample
    points_vector[l] = reference

In [6]:
# Cargamos el iterador para poder usarlos como variable de la función
xls = pd.ExcelFile("C:/Users/cdedi/Desktop/numpy_parser.xlsx")
coord = xls.parse(index_col=None, na_values=['NA'])
coord.columns = coord.columns.str.replace(' ', '')
# Creamos un vector de tamaño del total de CMH analizadas
volume_vector = [0] * len(coord)
array_vector = [0] * len(coord)
age_vector = [0] * len(coord)
sample_vector = [0] * len(coord)
points_vector = [0] * len(coord)
centperim_vector = [0] * len(coord)
# Crea un contador para iniciar el conteo de células
counter = 0
# Itera sobre cada célula
for j in range(0, len(coord),1):
    hsc_segmentation(coord.at[j,"Status"], coord.at[j,"Sample"], coord.at[j,"Reference"], coord.at[j, "PositionX"], coord.at[j, "PositionY"], 12, counter)
    counter += 1

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 4 Points 1 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 4 Points 1 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 4 Points 1 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image us

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 4 Points 10 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 4 Points 11 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 4 Points 11 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 8 Points 11 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 8 Points 11 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Femur 8 Points 13 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 1 Points 2 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 1 Points 2 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 1 Points 2 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the im

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 1 Points 11 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 1 Points 11 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 1 Points 11 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 6 Points 2 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 6 Points 2 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 6 Points 3 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the im

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 6 Points 12 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 6 Points 12 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Aged Humerus 6 Points 12 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Young Femur 7 Points 5 red_0.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Young Femur 7 Points 5 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Young Femur 7 Points 5 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image

<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Young Humerus 4 Points 4 red_1.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)
<ipython-input-5-4061402cca40>:38: UserWarning: C:/Users/cdedi/Desktop/Otsu_segmented/Young Humerus 4 Points 4 red_2.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  skimage.io.imsave("C:/Users/cdedi/Desktop/Otsu_segmented/" + status + " " + sample + " " + reference + " red_" + str(k) + ".tif", segmented)


In [7]:
# Crea un dataframe para almacenar características
hsc_dataset = pd.DataFrame(list(zip(age_vector, sample_vector, points_vector, array_vector, volume_vector,area_vector)),  columns =['Status', 'Sample', 'Reference', "Numpy","Volume","CentralArea"])
hsc_dataset.to_excel("C:/Users/cdedi/Desktop/working_dataset.xlsx", index=False)
# Crea un dataframe para almacenar características
classificator_dataset = pd.DataFrame(list(zip(age_vector, volume_vector, area_vector)),  columns =['Status',"Volume","CentralArea"])
classificator_dataset = classificator_dataset.sample(frac=1)
classificator_dataset.to_excel("C:/Users/cdedi/Desktop/classification.xlsx", index=False)